<h1>Matt's eCFR API Demonstrator</h1>
<p>An intro to Jupyter Notebooks</p>
<p><i>This is a very important subtitle, to be updated later</i></p>

In [12]:
# Do my imports - pandas for data analysis, requests for HTTPS REST API requests, json for, well, javascript object notation, duh
import pandas as pd
import requests as rq
import json as json

# Set my constants
base_url = "https://www.ecfr.gov/api/"
titles_url = base_url + "versioner/v1/titles.json"
agencies_url = base_url + "admin/v1/agencies.json"
data_path = "/workspaces/codespaces-jupyter/notebooks/ecfr_data/"
ecfr_titles = {'titles':[]}


print("Welcome to the MRWeCFR with base_url of "+base_url)

# Psuedocode
# 1. Let's see if we have a list of recent titles (less than 30 days old) in our titles.json file
# 2. If we do, let's see if we have the full and amendments of similar age, else, try and refresh titles.json 
# 3. If we are good with source data, skip to analysis, else, try and refresh the eCFR json details 
# 4. Analysis - let's load some basic semantic checks (wordcount, reading level) and some change trackers (count of changes, frequency over time)

try:
    # lets see if we have some files
    print("Let\'s do this - reading the file now")
    with open(data_path+'titles.json') as mytitlesfile:
        titles_json = json.load(mytitlesfile)


    #"up_to_date_as_of": "2025-07-03"

except FileNotFoundError:
    # oh dang. no file
    print("Whoops - file not found. Ain\'t got nothin\' sorry")

else:
    # no exception occurred - woohoo
    print("Nothing to see here but these titles \n")
    print(titles_json)

finally:
    # do this always
    print("finished risky job")


type(titles_json)

Welcome to the MRWeCFR with base_url of https://www.ecfr.gov/api/
Let's do this - reading the file now
Nothing to see here but these titles 

{'titles': [{'number': 1, 'name': 'General Provisions', 'latest_amended_on': '2022-12-29', 'latest_issue_date': '2024-05-17', 'up_to_date_as_of': '2025-07-03', 'reserved': False}, {'number': 2, 'name': 'Federal Financial Assistance', 'latest_amended_on': '2025-01-03', 'latest_issue_date': '2025-05-15', 'up_to_date_as_of': '2025-07-03', 'reserved': False}, {'number': 3, 'name': 'The President', 'latest_amended_on': '2015-03-17', 'latest_issue_date': '2024-05-17', 'up_to_date_as_of': '2025-07-03', 'reserved': False}, {'number': 4, 'name': 'Accounts', 'latest_amended_on': '2024-07-18', 'latest_issue_date': '2024-07-18', 'up_to_date_as_of': '2025-07-03', 'reserved': False}, {'number': 5, 'name': 'Administrative Personnel', 'latest_amended_on': '2025-06-24', 'latest_issue_date': '2025-06-24', 'up_to_date_as_of': '2025-07-03', 'reserved': False}, {'num

dict